In [4]:
import pandas as pd

In [5]:
df = pd.read_csv(r'C:\Users\kazi.shahid\Downloads\CFPB Consumer Complaints Dataset\complaints.csv')

In [6]:
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2020-07-06,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,FL,346XX,NaN,Other,Web,2020-07-06,Closed with explanation,Yes,NaN,3730948
1,2025-12-27,Credit reporting or other personal consumer re...,Credit reporting,Problem with a company's investigation into an...,Their investigation did not fix an error on yo...,NaN,NaN,Experian Information Solutions Inc.,IN,464XX,NaN,NaN,Web,2025-12-27,In progress,Yes,NaN,18309451
2,2025-11-25,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,NaN,"EQUIFAX, INC.",GA,30311,NaN,Consent not provided,Web,2025-11-25,Closed with explanation,Yes,NaN,17487731
3,2025-10-14,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information is missing that should be on the r...,My credit report contains incorrect and mislea...,NaN,"EQUIFAX, INC.",TX,75062,NaN,Consent provided,Web,2025-10-14,Closed with non-monetary relief,Yes,NaN,16558024
4,2025-10-26,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information belongs to someone else,Be sure that disputed accounts are taken down ...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",CA,93619,NaN,Consent provided,Web,2025-10-26,Closed with non-monetary relief,Yes,NaN,16826857


In [7]:
print(f"{len(df):,}")

13,382,974


In [8]:
df['Product'].value_counts().reset_index(name='num_instances')

,Product,num_instances
0,Credit reporting or other personal consumer re...,8232642
1,"Credit reporting, credit repair services, or o...",2163844
2,Debt collection,991195
3,Mortgage,438565
4,Checking or savings account,339128
5,Credit card,283566
6,Credit card or prepaid card,206368
7,"Money transfer, virtual currency, or money ser...",167067
8,Credit reporting,140429
9,Student loan,120273


In [9]:
df['Sub-product'].value_counts().reset_index(name='num_instances')

,Sub-product,num_instances
0,Credit reporting,10341380
1,I do not know,359012
2,Checking account,333051
3,General-purpose credit card or charge card,332782
4,Other debt,178288
...,...,...
81,Traveler’s/Cashier’s checks,88
82,Student prepaid card,54
83,Transit card,37
84,Tax refund anticipation loan or check,35


In [10]:
print(f"Dates range from {min(df['Date received'])} to {max(df['Date received'])}")

Dates range from 2011-12-01 to 2026-01-31


# Stratified Sample

A proportional stratified sampling strategy was implemented to preserve the joint distribution of complaint types across time and region while reducing computational burden.

I wanted to do stratification across:
- Product
- Year-Quarter
- Geography (likely State)

[...]

## Step 1: Creating Year-Quarter

In [11]:
df['Date received'] = pd.to_datetime(df['Date received'], errors='coerce')

df['year_quarter'] = (
    df['Date received']
      .dt.to_period('Q')
      .astype(str)
)

This gives values like '2021Q1', '2022Q3', etc.

## Step 2: Defining Geography Level

Needed to decide on the level of geography (e.g., state-level / region-level, etc.). 50 states may seem too sparse, OTOH regions may seem too few. I started with the more granular approach: grouping by states.

In [12]:
df['geo'] = df['State']

## Step 3: Dropping Missing Key Fields

Stratified sampling only works cleanly if strata are defined, so I dropped rows that had any of the specified strata info missing.

In [13]:
df_strata = df.dropna(subset=['Product', 'year_quarter', 'geo'])

## Step 4: Creating a Strata Key

Next, I ensured that each unique combination is one stratum, so that the stratifying works next.

In [14]:
df_strata['stratum'] = (
    df_strata['Product'].astype(str) + '|' +
    df_strata['year_quarter'].astype(str) + '|' +
    df_strata['geo'].astype(str)
)

C:\Users\kazi.shahid\AppData\Local\Temp\ipykernel_7760\1737723053.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_strata['stratum'] = (


## Step 5: Computing Stratum Sizes

In [15]:
stratum_counts = df_strata['stratum'].value_counts()

total_rows = len(df_strata)

## Step 6: Proportional Stratified Sampling

For a target size (say 500,000): n_stratum = ( N_stratum / N_total ) x N_target

In [16]:
import numpy as np

def stratified_sample(df, target_size, random_state=42):
    df = df.copy()
    
    # Calculate proportions
    stratum_counts = df['stratum'].value_counts()
    total = len(df)
    
    # Target sample per stratum
    stratum_sample_sizes = (
        (stratum_counts / total) * target_size
    ).round().astype(int)
    
    # Merge sizes back
    sample_map = stratum_sample_sizes.to_dict()
    df['sample_size'] = df['stratum'].map(sample_map)
    
    # Sample within each stratum
    sampled_df = (
        df.groupby('stratum', group_keys=False)
          .apply(lambda x: x.sample(
              n=min(len(x), sample_map[x.name]),
              random_state=random_state
          ))
    )
    
    return sampled_df.drop(columns=['sample_size'])


## Step 7: Generating Samples

I wanted to start with a smaller sample (e.g., 300k; still stratified) so it's computationally practical to analyze. However, considering a lot of them may be missing review text -- which I have to drop eventually as I will not be able to perform text-based analysis on them -- that may reduce the sample size even further, I decided to generate a 500k-size sample as well.

In [17]:
# 500k sample generation
df_500k = stratified_sample(df_strata, 500_000)

# 300k sample generation
df_300k = stratified_sample(df_strata, 300_000)

C:\Users\kazi.shahid\AppData\Local\Temp\ipykernel_7760\1242168485.py:22: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(
C:\Users\kazi.shahid\AppData\Local\Temp\ipykernel_7760\1242168485.py:22: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(


## Step 8: Validating Stratification

Now I want to verify that the population stratification was preserved in the samples.

In [18]:
def compare_distribution(original, sample, col):
    orig_dist = original[col].value_counts(normalize=True)
    sample_dist = sample[col].value_counts(normalize=True)
    return pd.concat([orig_dist, sample_dist], axis=1, keys=['original', 'sample'])

In [19]:
compare_distribution(df_strata, df_500k, 'Product')
compare_distribution(df_strata, df_500k, 'year_quarter')
compare_distribution(df_strata, df_500k, 'geo')

,original,sample
geo,,
FL,0.136486,0.136753
TX,0.129970,0.130219
CA,0.102102,0.102307
GA,0.074128,0.074246
NY,0.060524,0.060612
...,...,...
AA,0.000011,0.000006
FM,0.000009,NaN
MP,0.000007,0.000002


One issue we can observe is that, some geo codes (e.g., FM, MH, PW, etc.) had small representation in the original dataset already, and resulted in zero representation in the stratified sample. Plus, 63 geo types may seem to sparse, EDA on them may not provide useful, directional insights. So I revert to region as geography.

For US datasets, the standard and defensible approach is U.S. Census Bureau Regions (Census Regions:

**1. Northeast**
- CT, ME, MA, NH, RI, VT
- NJ, NY, PA

**2. Midwest**
- IL, IN, MI, OH, WI
- IA, KS, MN, MO, NE, ND, SD

**3. South**
- DE, FL, GA, MD, NC, SC, VA, DC, WV
- AL, KY, MS, TN
- AR, LA, OK, TX

**4. West**
- AZ, CO, ID, MT, NV, NM, UT, WY
- AK, CA, HI, OR, WA

**5. Others**
- For any other values in the State column (e.g., any US territories outside of the states listed above)

In [20]:
northeast = ['CT','ME','MA','NH','RI','VT','NJ','NY','PA']
midwest   = ['IL','IN','MI','OH','WI','IA','KS','MN','MO','NE','ND','SD']
south     = ['DE','FL','GA','MD','NC','SC','VA','DC','WV',
             'AL','KY','MS','TN','AR','LA','OK','TX']
west      = ['AZ','CO','ID','MT','NV','NM','UT','WY','AK','CA','HI','OR','WA']

def map_region(state):
    if state in northeast:
        return 'Northeast'
    elif state in midwest:
        return 'Midwest'
    elif state in south:
        return 'South'
    elif state in west:
        return 'West'
    else:
        return 'Other'

df['region'] = df['State'].apply(map_region)

Then I dropped rows without valid region (as opposed to, say, labeling them as 'Others' -- considering we are generating a large sample anyway).

In [21]:
# Drop rows without valid region
df = df.dropna(subset=['region'])

Next I verified distribution to see the balance.

In [22]:
df['region'].value_counts(normalize=True)

region
South        0.538654
West         0.162023
Northeast    0.159421
Midwest      0.131151
Other        0.008751
Name: proportion, dtype: float64

South region seems to be comprising slightly over half of the dataset, with the rest 3 regions taking up about evenly (13-16% each).

Next, I revert to using region as geography marker.

In [23]:
df['year_quarter'] = pd.to_datetime(df['Date received']).dt.to_period('Q').astype(str)

df['stratum'] = (
    df['Product'].astype(str) + "|" +
    df['year_quarter'].astype(str) + "|" +
    df['region'].astype(str)
)


Just as a sanity check, I checked how many unique strata I now have. Per my rough estimation from **21 products**, **~40 quarters** (e.g., 2016–2025 → ~40 quarters), and **4 regions**, I should get **~3,360 unique strata**.

In [24]:
num_unique_strata = df['stratum'].nunique()
print(f"Total number of unique strata: {num_unique_strata:,}")

Total number of unique strata: 2,752


In [25]:
strata_counts = df.groupby(['Product', 'year_quarter', 'region']).size()

strata_counts.describe()

count      2752.000000
mean       4862.999273
std       34541.293086
min           1.000000
25%         140.000000
50%         533.000000
75%        1666.250000
max      917323.000000
dtype: float64

As we can see from above:
- Half of the strata have ≥ 847 observations, this is very healthy.
- That massive std (38k) and max (917k) tell us we have a heavy long tail (likely pertaining to Credit reporting, Debt collection, Mortgage, more recent quarters, etc.). This is normal for CFPB; not necessarily hinting at a flaw but rather reflecting real-world complaint distribution.

In [26]:
print(f"Number of strata with fewer than 20 samples: {(strata_counts < 20).sum():,} out of {num_unique_strata:,} (i.e., {((strata_counts < 20).sum()/num_unique_strata):.1}%).")

Number of strata with fewer than 20 samples: 195 out of 2,752 (i.e., 0.07%).


Seems reasonable enough, as there are (a) not too high number of unique strata that'd make them too granular for analyses, and (b) only a tiny fraction of strata (0.02%) are too small in size themselves, this is negligible.

# Final Sample

Based on the above, I decided to do the following:

1. Stratification by:
- Product
- year-quarter
- region

2. Proportional allocation across strata

3. Final sample size = 300,000 (for now)

## Step 1: Defining Strata

In [27]:
strata_cols = ['Product', 'year_quarter', 'region']

## Step 2: Computing Proportional Allocation (Adjusting for Rounding Drift)

In [28]:
# Target sample size
TARGET_N = 300_000

# Get stratum size
strata_counts = df.groupby(strata_cols).size().reset_index(name='count')

# Compute proportions
total_records = len(df)

strata_counts['prop'] = strata_counts['count'] / total_records
strata_counts['sample_n'] = (strata_counts['prop'] * TARGET_N).round().astype(int)

# Adjust for Rounding Drift
difference = TARGET_N - strata_counts['sample_n'].sum()

if difference != 0:
    strata_counts.loc[strata_counts.sample(difference).index, 'sample_n'] += 1


## Step 3: Merging Back & Creating Sample

In [29]:
# Merge sample sizes into main df
df = df.merge(
    strata_counts[strata_cols + ['sample_n']],
    on=strata_cols,
    how='left'
)

In [30]:
# Sample within each stratum
df_sample_300k = (
    df.groupby(strata_cols, group_keys=False)
      .apply(lambda x: x.sample(
          n=min(len(x), x['sample_n'].iloc[0]),
          random_state=42
      ))
)

C:\Users\kazi.shahid\AppData\Local\Temp\ipykernel_7760\1137883942.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(


## Step 4: Verifying Sample Size & Distribution

In [31]:
# Verify sample size = Original target sample size
print(f"Sample size: {len(df_sample_300k):,}")

Sample size: 300,000


In [32]:
# Verify distribution
df_sample_300k.groupby(strata_cols).size().describe()

count     2543.000000
mean       117.970901
std        804.840619
min          1.000000
25%          5.000000
50%         15.000000
75%         40.000000
max      20563.000000
dtype: float64

# Conclusion

To summarize: I performed proportional stratified sampling across Product × Year-Quarter × Region to preserve the joint distribution of complaint type, temporal trends, and geographic variation while reducing dataset size for computational efficiency.

By creating the above sample, I have managed to achieve that the sample:
- Preserves real-world distribution
- Avoids bias toward dominant products
- Maintains temporal structure
- Maintains geographic representation
- Fully reproducible (random_state=42)

In [35]:
df_sample_300k.to_parquet(
    "data/processed/cfpb_sample_300k.parquet",
    index=False
)